In [ ]:
%load_ext autoreload
%autoreload 2


import os, sys

import torch
import torchio as tio
from ipywidgets import interact
import matplotlib.pyplot as plt

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
from data_processing import *

In [ ]:
subject_loader = ComposeLoaders([
ImageLoader(glob_pattern="whole_roi.*", image_name="whole_roi", image_constructor=tio.LabelMap,
                label_values={"left_whole": 1, "right_whole": 2}
                ),
])

cohorts = {'all': RequireAttributes(['whole_roi'])}
dataset_path = "X:/Datasets/Diffusion_MRI/"
dataset = SubjectFolder(root=dataset_path, subject_path="subjects", cohorts=cohorts, subject_loader=subject_loader)

In [ ]:
from skimage.measure import marching_cubes
import meshio

image_name = 'whole_roi'
for subject in dataset:
    x = subject[image_name].data[0]

    W, H, D = x.shape
    half_W = W // 2

    left_x = x[:half_W]
    right_x = x[half_W:]

    left_verts, left_faces, left_normals, left_values = marching_cubes(left_x.numpy())
    right_verts, right_faces, right_normals, right_values = marching_cubes(right_x.numpy())

    left_mesh = meshio.Mesh(left_verts, [("triangle", left_faces)])
    right_mesh = meshio.Mesh(right_verts, [("triangle", right_faces)])

    out_path = os.path.join(dataset_path, "shape", subject['name'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    left_mesh.write(os.path.join(out_path, f"left_{image_name}.obj"))
    right_mesh.write(os.path.join(out_path, f"right_{image_name}.obj"))

In [ ]:
import pygalmesh

image_name = 'whole_roi'
for subject in dataset:
    print(subject['name'])
    x = subject[image_name].data[0]

    W, H, D = x.shape
    half_W = W // 2

    left_x = (x[:half_W] > 0).to(torch.uint8)
    right_x = (x[half_W:] > 0).to(torch.uint8)
    
    # See 4.2.2 https://doc.cgal.org/latest/Mesh_3/index.html
    # and https://github.com/nschloe/pygalmesh/blob/dc2bd6c920c36cf27f5c89738bbe9488e3c4fe43/pygalmesh/main.py#L428
    params = {'facet_distance': 1., 'cell_size': 1.}
    voxel_size = (1., 1., 1.)
    left_mesh = pygalmesh.generate_from_array(left_x.numpy(), voxel_size, **params)
    right_mesh = pygalmesh.generate_from_array(right_x.numpy(), voxel_size, **params)

    out_path = os.path.join(dataset_path, "pygalmesh", subject['name'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    left_mesh.write(os.path.join(out_path, f"left_{image_name}.vtk"))
    right_mesh.write(os.path.join(out_path, f"right_{image_name}.vtk"))

In [ ]:
import numpy as np
import pyvista as pv
import pygalmesh
import skfmm

subject_names = [subject['name'] for subject in dataset.all_subjects]

@interact(subject_name=subject_names)
def show(subject_name):
    subject = dataset[subject_name]
    
    #@interact(facet_distance=(0., 2.), cell_size=(0., 2.))
    #def choose_params(facet_distance, cell_size):
    x = subject['whole_roi'].data[0]
    x[x == 0] = -1
    x = skfmm.distance(x)
    x = torch.from_numpy(x)

    W, H, D = x.shape
    half_W = W // 2

    left_x = x[:half_W]
    right_x = x[half_W:]

    params = {
        'edge_size': 0.5,
        'facet_angle': 30., 
        'facet_size': 1.5, 
        'facet_distance': 2., 
        'cell_radius_edge_ratio': 2., 
        'cell_size': 1.0,
    }
    voxel_size = (1., 1., 1.)
    left_mesh = pygalmesh.generate_from_array(left_x.numpy(), voxel_size, **params)
    right_mesh = pygalmesh.generate_from_array(right_x.numpy(), voxel_size, **params)

    plotter = pv.Plotter(notebook=True)
    mesh = pv.wrap(left_mesh)
    plotter.add_mesh(mesh)
    plotter.show(jupyter_backend='ipygany')

In [ ]:
from dual_contouring.dual_contour_3d import dual_contour_3d
from dual_contouring.utils_3d import V3
import numpy as np

def clamp(v, vmin, vmax):
    return min(max(v, vmin), vmax)

def dual_contour_array(arr):
    arr_grad = np.stack(np.gradient(left_x))
    
    def f(x, y, z):
        x = clamp(x, 0, arr.shape[0] - 1)
        y = clamp(y, 0, arr.shape[1] - 1)
        z = clamp(z, 0, arr.shape[2] - 1)
        return arr[x, y, z]
    
    def f_grad(x, y, z):
        x, y, z = int(x), int(y), int(z)
        return V3(arr_grad[0, x, y, z], arr_grad[1, x, y, z], arr_grad[2, x, y, z])
    
    args = dict(xmin=0, ymin=0, zmin=0, xmax=arr.shape[0], ymax=arr.shape[1], zmax=arr.shape[2])
    mesh = dual_contour_3d(f, f_grad, **args)
    return mesh
    
image_name = 'whole_roi'

subject = dataset[0]
x = subject[image_name].data[0]

W, H, D = x.shape
half_W = W // 2

left_x = x[:half_W] 

left_mesh = dual_contour_array(left_x)
print(left_mesh)

In [ ]:
import meshio

verts = [[v.x, v.y, v.z] for v in left_mesh.verts]
faces = [[f.v1-1, f.v2-1, f.v3-1, f.v4-1] for f in left_mesh.faces]

cells = [('quad', faces)]

mesh = meshio.Mesh(verts, cells)
mesh.write("foo.obj")

In [ ]:
import meshio
import pyvista as pv

verts = [[v.x, v.y, v.z] for v in left_mesh.verts]
faces = [[f.v1, f.v2, f.v3, f.v4] for f in left_mesh.faces]

cells = [('quad', faces)]

plotter = pv.Plotter(notebook=True)
mesh = pv.wrap(meshio.Mesh(verts, cells))
plotter.add_mesh(mesh)
plotter.show(jupyter_backend='ipygany')

In [1]:
  from vedo import dataurl, Volume
from vedo.applications import IsosurfaceBrowser

vol = Volume(dataurl+'head.vti')

plt = IsosurfaceBrowser(vol, c='gold') # Plotter instance

plt.show(axes=7, bg2='lb').close()

In [ ]:
import CGAL
from CGAL import CGAL_Surface_mesher
from CGAL.CGAL_Kernel import Point_3, Sphere_3, ORIGIN
from CGAL.CGAL_Surface_mesher import \
    Surface_mesh_default_triangulation_3, \
    Surface_mesh_default_criteria_3, \
    Complex_2_in_triangulation_3, \
    make_surface_mesh, \
    NON_MANIFOLD_TAG


def sphere_function(p):
    return p.x() ** 2 + p.y() ** 2 + p.z() ** 2 - 1;


def main():
    t3 = Surface_mesh_default_triangulation_3()
    c2t3 = Complex_2_in_triangulation_3(t3)
    
    # Import missing for Surface_3
    surface = Surface_3(sphere_function, Sphere_3(Point_3(0., 0., 0.), 2.))
    
    criteria = Surface_mesh_default_criteria_3(30., 0.1, 0.1)
    
    make_surface_mesh(c2t3, surface, criteria, NON_MANIFOLD_TAG)
    


In [ ]:
CGAL_Surface_mesher.__dict__